# NBA Data Analysis for Prediction Model

## Table of Contents <a id="table_of_contents"></a>

0. [Import Libraries](#imports)
1. [Import Data](#import_data)
2. Data Exploration
3. Data Cleaning
4. Data Preparation
5. Benchmark Model
6. Visualization ?

## 0. Importing Libraries <a id="imports"></a>

[Back to top](#table_of_contents)

In [1]:
!pip install opendatasets

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/geometry-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
pip install pypyodbc

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/geometry-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib
from matplotlib import pyplot as plt
import opendatasets as od
import os

## 1. Import Data <a id="import_data"></a>

[Back to top](#table_of_contents)

In [7]:
#create connection to database file in Kaggle
dataset = "https://www.kaggle.com/datasets/wyattowalsh/basketball"
od.download(dataset)

Skipping, found downloaded files in "./basketball" (use force=True to force download)


In [8]:
# 1.Connect to SQL database
connection = sql.connect('basketball/nba.sqlite')
# Test SQL connectivity to database by writing a SQL query
test_query = "SELECT * FROM game"
# Load the data into a pandas DataFrame
df = pd.read_sql_query(test_query, connection)
# Display the DataFrame
print(df)
# Close Connection if necessary
## connection.close()

      season_id team_id_home team_abbreviation_home           team_name_home  \
0         21946   1610610035                    HUS          Toronto Huskies   
1         21946   1610610034                    BOM        St. Louis Bombers   
2         21946   1610610032                    PRO  Providence Steamrollers   
3         21946   1610610025                    CHS            Chicago Stags   
4         21946   1610610028                    DEF          Detroit Falcons   
...         ...          ...                    ...                      ...   
65693     42022   1610612748                    MIA               Miami Heat   
65694     42022   1610612748                    MIA               Miami Heat   
65695     42022   1610612743                    DEN           Denver Nuggets   
65696     32022   1610616834                    LBN              Team LeBron   
65697     32022   1610616834                    LBN              Team LeBron   

          game_id            game_date 